<a href="https://colab.research.google.com/github/SamuelBFG/DL-studies/blob/master/1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Instructions**
You need to have an account on [Kaggle](https://www.kaggle.com/).

The dataset for this activity is in the following [link](https://www.kaggle.com/ceciliala/caltech-101).

You'll need to follow the steps on this [video](https://www.youtube.com/watch?v=tGw-ZACouik&feature=youtu.be) (untill minute 3:52) in order to download and unzip the dataset in your Google Colab enviroment. 

**Please don't forget to change your runtime type to GPU.**

In [1]:
#install depencencies 
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samuelbfg","key":"0720c91e1f13c46c81f9cf850f621e4e"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Change the permission
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d ceciliala/caltech-101

 98% 113M/115M [00:02<00:00, 33.6MB/s] 
100% 115M/115M [00:02<00:00, 57.0MB/s]


In [5]:
from zipfile import ZipFile
file_name = "caltech-101.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [7]:
!pip install tqdm

In [6]:
#If you are working in anaconda enviroment please use the following conda install command and comment the pip line
#!conda install --yes -c conda-forge python-annoy

#If you are using Google Colab use the pip command for instalation
!pip install annoy -U

     |████████████████████████████████| 655kB 4.7MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390362 sha256=73d5a16210b0e85ef3a06e322e154cc30b07f751d5a7f199da03e3c4ccddeebb
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [8]:
#If you are working in anaconda enviroment please use the conda install command
#!conda install --yes nodejs

#If you are using Google Colab use the pip command for instalation
!pip install nodejs

  Created wheel for nodejs: filename=nodejs-0.1.1-cp36-none-any.whl size=3494 sha256=b8187a614a07f51335d87b0cbce8e381bf2775bd243d81a7df0f7d79f6ab1147
  Stored in directory: /root/.cache/pip/wheels/08/dd/8a/460c064dd7665365011f49f5431df98d59970d3b8a1599436b
  Created wheel for optional-django: filename=optional_django-0.1.0-cp36-none-any.whl size=9981 sha256=a2852180d6deb3614b879feaeb049d07e83a1d88dce43acdc4ce8fa68a9d9877
  Stored in directory: /root/.cache/pip/wheels/b9/0f/04/efd2745af85731eae4a16fd4c33a95d3ba243b5ad9ab43a5c7
Successfully built nodejs optional-django


In [9]:
!pip install ipywidgets

#If you are working in the anaconda enviroment uncomment the two following lines
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

#If you are working with Google Colab
!pip install jupyter-contrib-nbextensions enable --py widgetsnbextension
!pip install jupyterlab
!jupyter labextension install @jupyter-widgets/jupyterlab/google-drive


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --py (--pypi-url, --python-version?)
     |████████████████████████████████| 7.8MB 5.3MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
ERROR: nbclient 0.5.0 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
  Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
An error occured.
ValueError: Please install nodejs >=10.0.0 before continuing. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /tmp/jupyterlab-debug-w3fq3zwk.log


## Feature Extraction

In [10]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm.notebook import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

We will define a helper function that allows us to choose any pretrained model with all the necessary details for our experiments.

In [11]:
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1,
                          alpha=1)
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

Now, let's put our function to use.

In [12]:
model_architecture = 'resnet'
model = model_picker(model_architecture)

94773248/94765736 [==============================] - 1s 0us/step


In [13]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [14]:
#select the cat.jpg image
from google.colab import files
files.upload()

Saving cat.jpg to cat.jpg


{'cat.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x07hExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\t\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x00z\x01\x10\x00\x02\x00\x00\x00\t\x00\x00\x00\x80\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\x8a\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\x92\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00\x07\x00\x00\x00\x9a\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\xa2\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xb6\x88%\x00\x04\x00\x00\x00\x01\x00\x00\x06f\x00\x00\x00\x00Apple\x00iPhone 7\x00\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x0110.3.3\x00\x002017:08:17 18:09:36\x00\x00 \x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x02<\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x02D\x88"\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x88\'\x00\x03\x00\x00\x00\x01\x00@\x00\x00\x90\x00\x00\x07\x00\x00\x00\x040221\x90\x03\x00\x02\x00\x00\x00\x14\x00\x00\x02L\x90\x04\x00\x02\x00\x00\x00\x14\x00\x00\x02`\x91\x01\x00\x07

In [15]:
features = extract_features("cat.jpg", model)
print(len(features))

2048


In [16]:
%timeit features = extract_features("cat.jpg", model)

10 loops, best of 3: 87.3 ms per loop


In [17]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

In [18]:
# add the path to your dataset
root_dir = "caltech101"
filenames = sorted(get_file_list(root_dir))

feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

In [19]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict_generator(generator, num_epochs)
end_time = time.time()

Found 8677 images belonging to 101 classes.
Instructions for updating:
Please use Model.predict, which supports generators.


In [20]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 2048)
Time taken in sec =  20.193992614746094


In [21]:
!mkdir data

In [22]:
filenames = [root_dir + '/' + s for s in generator.filenames]

In [23]:
pickle.dump(generator.classes, open('data/class_ids-caltech101.pickle','wb'))
pickle.dump(filenames, open('data/filenames-caltech101.pickle', 'wb'))
pickle.dump(feature_list, open('data/features-caltech101-' + model_architecture + '.pickle', 'wb'))

In [24]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [25]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)

In [26]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')

Found 8677 images belonging to 101 classes.


In [27]:
def model_maker():
    base_model = ResNet50(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [28]:
model_finetuned = model_maker()
model_finetuned.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model_finetuned.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
136/136 [==============================] - 95s 699ms/step - loss: 2.8886 - acc: 0.3688
Epoch 2/10
136/136 [==============================] - 95s 696ms/step - loss: 1.6362 - acc: 0.6031
Epoch 3/10
136/136 [==============================] - 94s 693ms/step - loss: 1.2241 - acc: 0.6735
Epoch 4/10
136/136 [==============================] - 94s 691ms/step - loss: 1.0346 - acc: 0.7201
Epoch 5/10
136/136 [==============================] - 94s 691ms/step - loss: 0.8898 - acc: 0.7597
Epoch 6/10
136/136 [==============================] - 94s 690ms/step - loss: 0.8413 - acc: 0.7677
Epoch 7/10
136/136 [==============================] - 94s 692ms/step - loss: 0.7724 - acc: 0.7799
Epoch 8/10
136/136 [==============================] - 94s 691ms/step - loss: 0.7419 - acc: 0.7883
Epoch 9/10
136/136 [==============================] - 94s 690ms/step - loss: 0.6706 - acc: 0.8017
Epoch 10/10
136/136 [=====================

In [29]:
model_finetuned.save("/content/data/model-finetuned.h5")

In [30]:
start_time = time.time()
feature_list_finetuned = []
feature_list_finetuned = model_finetuned.predict_generator(generator, num_epochs)
end_time = time.time()

for i, features_finetuned in enumerate(feature_list_finetuned):
    feature_list_finetuned[i] = features_finetuned / norm(features_finetuned)

feature_list = feature_list_finetuned.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 101)
Time taken in sec =  19.596739053726196


In [31]:
pickle.dump(
    feature_list,
    open("/content/data/features-caltech101-resnet-finetuned.pickle", 'wb'))

**Note:** Please download and store all the files allocated in the ***data*** folder because we'll be using it in other Chapter 4 notebooks. 

FUNÇÃO AUXILIAR PARA FAZER O DOWNLOAD DA PASTA DATA:

In [32]:
!zip -r /content/data.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/model-finetuned.h5 (deflated 8%)
  adding: content/data/features-caltech101-resnet.pickle (deflated 17%)
  adding: content/data/filenames-caltech101.pickle (deflated 90%)
  adding: content/data/class_ids-caltech101.pickle (deflated 99%)
  adding: content/data/features-caltech101-resnet-finetuned.pickle (deflated 4%)


In [33]:
from google.colab import files
files.download("/content/data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>